In [5]:
import numpy as np
import torch
from sklearn.linear_model import QuantileRegressor

In [4]:
beta = np.random.randn(10)
X = np.random.randn(10000 * 10).reshape(10000, 10)
Y = X @ beta + np.random.randn(10000)

In [ ]:
QuantileRegressor(quantile=0.5, alpha=0.0).fit(X, Y)

In [6]:
Xt = torch.as_tensor(X)
Yt = torch.as_tensor(Y)

In [112]:
beta_hat = torch.tensor(np.zeros(10), requires_grad=True)
beta0_hat = torch.tensor(0.0, requires_grad=True)

In [113]:
def quantile_loss(error, q):
    loss = q * torch.relu(error) + (1 - q) * torch.relu(-error)
    return loss

In [114]:
q = 0.8

In [115]:
quantile_loss(Yt - Xt @ beta_hat - beta0_hat, 0.8).mean()

tensor(1.1472, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [116]:
L = max(q, 1 - q)

optim = torch.optim.SGD([beta_hat, beta0_hat], lr=1 / L)

for i in range(20):
    loss = quantile_loss(Yt - Xt @ beta_hat - beta0_hat, q).mean()
    loss.backward()
    optim.step()
    optim.zero_grad()

In [117]:
loss

tensor(0.2799, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [118]:
(Yt - Xt @ beta_hat - beta0_hat > 0).float().mean()

tensor(0.1997)